In [51]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import sys
import json
import time
import datetime as dt
import requests
from dotenv import load_dotenv

TOKENS_PATH = "tokens.json"
TOKEN_ENDPOINT = "https://open.tiktokapis.com/v2/oauth/token/"

def load_env():
    load_dotenv()  # ローカル開発用。不要なら消してOK
    client_key    = os.getenv("CLIENT_KEY")
    client_secret = os.getenv("CLIENT_SECRET")
    if not client_key or not client_secret:
        print("ERROR: CLIENT_KEY / CLIENT_SECRET が環境変数にありません。", file=sys.stderr)
        sys.exit(1)
    return client_key, client_secret

def load_tokens(path=TOKENS_PATH):
    try:
        with open(path, "r", encoding="utf-8") as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"ERROR: {path} が見つかりません。まず callback.js で初回取得してください。", file=sys.stderr)
        sys.exit(1)

def save_tokens(tokens, path=TOKENS_PATH):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(tokens, f, ensure_ascii=False, indent=2)
    print(f"✔ 新しいトークンを {path} に保存しました。")

def refresh_access_token(client_key, client_secret, refresh_token):
    payload = {
        "client_key":    client_key,
        "client_secret": client_secret,
        "grant_type":    "refresh_token",
        "refresh_token": refresh_token
    }
    headers = { "Content-Type": "application/x-www-form-urlencoded" }

    resp = requests.post(TOKEN_ENDPOINT, data=payload, headers=headers, timeout=15)
    try:
        data = resp.json()
    except ValueError:
        print("ERROR: レスポンスが JSON ではありません:", resp.text, file=sys.stderr)
        sys.exit(1)

    if resp.status_code != 200 or data.get("error"):
        print("ERROR: リフレッシュに失敗しました:", json.dumps(data, ensure_ascii=False), file=sys.stderr)
        sys.exit(1)

    return data

def human_expiry(seconds_from_now):
    return dt.datetime.utcnow() + dt.timedelta(seconds=seconds_from_now)

def main():
    client_key, client_secret = load_env()
    old = load_tokens()

    if "refresh_token" not in old:
        print("ERROR: tokens.json に refresh_token が含まれていません。初回取得をやり直してください。", file=sys.stderr)
        sys.exit(1)

    print("現在の access_token は更新対象です。refresh_token を使って更新します…")

    new_tokens = refresh_access_token(client_key, client_secret, old["refresh_token"])

    # 便利情報を付け足して保存（任意）
    now = int(time.time())
    new_tokens["_fetched_at"]   = now
    new_tokens["_expire_at"]    = now + int(new_tokens.get("expires_in", 0))
    new_tokens["_expire_at_utc"] = human_expiry(int(new_tokens.get("expires_in", 0))).isoformat() + "Z"

    save_tokens(new_tokens)

    print("\n=== 新しいトークン ===")
    print(json.dumps(new_tokens, ensure_ascii=False, indent=2))

    print("\naccess_token 失効予定（UTC）:", new_tokens["_expire_at_utc"])

if __name__ == "__main__":
    main()


現在の access_token は更新対象です。refresh_token を使って更新します…
✔ 新しいトークンを tokens.json に保存しました。

=== 新しいトークン ===
{
  "access_token": "act.lYasXfQYmPeLxOTtVZ8peQaBQZ8wmXW2QSPGvwhZfVAeOE1bzdXENdHYAGkN!5271.va",
  "expires_in": 86400,
  "open_id": "-000DT7UI3F8wq1VMCRwCISsWdWDWjawYI2j",
  "refresh_expires_in": 31506440,
  "refresh_token": "rft.79tPRDRUtASmEiNYbzIC93VgFx1NQhlp7KvcqybdULKTK8uKffCJzV8Hgm6Y!5269.va",
  "scope": "user.info.basic",
  "token_type": "Bearer",
  "_fetched_at": 1753621798,
  "_expire_at": 1753708198,
  "_expire_at_utc": "2025-07-28T13:09:58.126757Z"
}

access_token 失効予定（UTC）: 2025-07-28T13:09:58.126757Z


In [37]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import sys
import json
import requests
from urllib.parse import urlencode
from dotenv import load_dotenv

#── 環境変数ロード ──────────────────────────────────
load_dotenv()  # ローカル開発時のみ .env を読み込む

CLIENT_KEY    = os.getenv("CLIENT_KEY")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
REDIRECT_URI  = os.getenv("REDIRECT_URI")
SCOPES        = ["user.info.basic"]
NETLIFY_CALLBACK_URL = "https://tiktoker-master.netlify.app/.netlify/functions/callback"

if not all([CLIENT_KEY, CLIENT_SECRET, REDIRECT_URI]):
    print("ERROR: CLIENT_KEY, CLIENT_SECRET, REDIRECT_URI を環境変数で設定してください。", file=sys.stderr)
    sys.exit(1)

#── 1) 認可 URL を組み立てて表示 ────────────────────────
auth_params = {
    "client_key":    CLIENT_KEY,
    "redirect_uri":  REDIRECT_URI,
    "response_type": "code",
    "scope":         " ".join(SCOPES),
    "state":         "state123",  # 任意の CSRF 対策文字列
}
auth_url = "https://www.tiktok.com/v2/auth/authorize/?" + urlencode(auth_params)
print("\n▶ TikTok 認可 URL（ブラウザで開いて許可してください）:\n")
print(auth_url)
print("\n---\n")

#── 2) 認可コードの入力 ───────────────────────────────
code = input("上記 URL で許可後、ブラウザのアドレスバーに表示される code の値を入力してください: ").strip()
if not code:
    print("ERROR: 認可コード(code) が入力されませんでした。", file=sys.stderr)
    sys.exit(1)

#── 3) Netlify Functions の callback を呼び出してトークン取得 ───────────────
def fetch_token(code):
    try:
        resp = requests.get(NETLIFY_CALLBACK_URL, params={"code": code}, timeout=30)
        resp.raise_for_status()
        data = resp.json()
    except requests.RequestException as e:
        print("HTTP エラー:", e, file=sys.stderr)
        sys.exit(1)
    except ValueError:
        print("レスポンスがJSONではありません:", resp.text, file=sys.stderr)
        sys.exit(1)
    return data

token_json = fetch_token(code)

#── 4) レスポンス表示 & ファイル保存 ─────────────────────
print("\n=== 取得したトークン情報 ===")
print(json.dumps(token_json, ensure_ascii=False, indent=2))

with open("tokens.json", "w", encoding="utf-8") as f:
    json.dump(token_json, f, ensure_ascii=False, indent=2)
print("\n✔ tokens.json に保存しました。")


▶ TikTok 認可 URL（ブラウザで開いて許可してください）:

https://www.tiktok.com/v2/auth/authorize/?client_key=sbawd6f9kgtu4fstl5&redirect_uri=https%3A%2F%2Ftiktoker-master.netlify.app%2F.netlify%2Ffunctions%2Fcallback&response_type=code&scope=user.info.basic&state=state123

---



上記 URL で許可後、ブラウザのアドレスバーに表示される code の値を入力してください:  RfxsMpeCtIYBDTfZi_Qp1IrdRT10d_56vftlb3zP23mu2nmf4bCW01hUnWVxDo_huUG-6wZATXSfVmX41M02ZIY6dorCkSePqRUbVOFAS9gD7k6gL_riuQA74U6X_jbTmKdcSG8z4BeoLu3xwKaAmpTjslLmefjL4YSP-acPAiI-Y6hjoBAnEm9MprBac8oSzsAdgB-5qt-nLmsnKDmqaD8qrfr00HXxljO6Yn7Jy3Y%2A2%215351



=== 取得したトークン情報 ===
{
  "error": "invalid_grant",
  "error_description": "Authorization code is expired.",
  "log_id": "20250727044602746A06628D08807105B2"
}

✔ tokens.json に保存しました。


In [ ]:
#!/usr/bin/env python3
import os, json, requests, webbrowser, sys
from urllib.parse import urlencode
from dotenv import load_dotenv

load_dotenv()                     # .env を読む（ローカル用）
REDIRECT_URI  = os.getenv("REDIRECT_URI")
CLIENT_KEY    = os.getenv("CLIENT_KEY")  # あれば認可 URL 生成に使用
SCOPES = ["user.info.basic"]

auth_url = "https://www.tiktok.com/v2/auth/authorize/?" + urlencode({
    "client_key": CLIENT_KEY or "",
    "redirect_uri": REDIRECT_URI,
    "response_type": "code",
    "scope": " ".join(SCOPES),
    "state": "init"
})
print("ブラウザで開きます:", auth_url)
webbrowser.open(auth_url, new=1)

redir = input("\nリダイレクト URL か code を貼り付け: ").strip()
code  = redir.split("code=")[-1] if "code=" in redir else redir
if not code:
    sys.exit("code 取得失敗")

resp = requests.get(f"{REDIRECT_URI}?code={code}", timeout=30)
token_json = resp.json()
if token_json.get("error"):
    sys.exit(token_json)

with open("tokens.json", "w", encoding="utf-8") as f:
    json.dump(token_json, f, ensure_ascii=False, indent=2)
print("✔ tokens.json 作成完了")

ブラウザで開きます: https://www.tiktok.com/v2/auth/authorize/?client_key=sbawd6f9kgtu4fstl5&redirect_uri=https%3A%2F%2Ftiktoker-master.netlify.app%2F.netlify%2Ffunctions%2Fcallback&response_type=code&scope=user.info.basic&state=init
